In [11]:
import string
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from collections import Counter
import nltk

# Téléchargement les ressources NLTK 
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
def preprocess_sentence(sentence, remove_stopwords=True, use_lemmatization=True):
    """
    Prétraitement d'une phrase :
    - minuscules
    - suppression ponctuation
    - suppression stopwords
    - lemmatisation
    Retourne la liste des mots prétraités.
    """
    words = []

    if sentence:
        # Minuscules + suppression ponctuation
        translator = str.maketrans("", "", string.punctuation)
        cleaned = sentence.lower().translate(translator)

        # Tokenisation
        words = cleaned.split()

        # Stopwords
        if remove_stopwords:
            stop_words = set(stopwords.words("english"))
            words = [w for w in words if w not in stop_words]

        # Lemmatisation
        if use_lemmatization:
            lemmatizer = WordNetLemmatizer()
            words = [lemmatizer.lemmatize(w) for w in words]

    return words


In [13]:
def get_related_forms(word):
    forms = set([word])
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            for deriv in lemma.derivationally_related_forms():
                forms.add(deriv.name())
    return forms

print(get_related_forms("happy"))  # {'happy', 'happiness'}


{'happy', 'felicitousness', 'felicity', 'happiness'}


In [14]:
def are_synonyms(word1, word2):
    """
    Vérifie si deux mots sont synonymes avec WordNet.
    Retourne True si oui, sinon False.
    """
    # Même mot → synonymes
    if word1 == word2:
        result = True
    else:
        synsets1 = wordnet.synsets(word1)
        synsets2 = wordnet.synsets(word2)
        result = False

        # Vérification des synonymes
        if synsets1 and synsets2:
            for syn1 in synsets1:
                for syn2 in synsets2:
                    if syn1 == syn2:
                        result = True
                        break
                if result:
                    break

    return result


In [15]:
def are_related(w1, w2):
    """
    Vérifie si deux mots sont identiques, synonymes ou formes dérivées.
    Retourne True si une des conditions est vérifiée, sinon False.
    """
    result = False

    # Même mot
    if w1 == w2:
        result = True
    else:
        # Vérifie s’ils sont synonymes
        synsets1 = wordnet.synsets(w1)
        for syn1 in synsets1:
            for lemma in syn1.lemmas():
                if lemma.name() == w2:
                    result = True
                    break
            if result:
                break

        # Vérifie s’ils partagent une forme dérivée
        if not result:
            related_w1 = get_related_forms(w1)
            related_w2 = get_related_forms(w2)
            if related_w1 & related_w2:  # intersection non vide
                result = True

    return result


In [16]:
def jaccard_similarity_sentences(sentence1, sentence2, remove_stopwords=True, use_lemmatization=True):
    """
    Calcule la similarité de Jaccard entre deux phrases 
    en tenant compte :
    - des synonymes
    - des formes dérivées (happy ↔ happiness)
    - des répétitions (Counter)
    Retourne un float entre 0 et 1
    """
    score = 0.0  # valeur par défaut

    try:
        # Prétraitement
        words1 = preprocess_sentence(sentence1, remove_stopwords, use_lemmatization)
        words2 = preprocess_sentence(sentence2, remove_stopwords, use_lemmatization)

        if words1 and words2:
            counter1 = Counter(words1)
            counter2 = Counter(words2)

            intersection_count = 0
            used_pairs = set()

            for w1 in counter1:
                for w2 in counter2:
                    if are_related(w1, w2) and (w1, w2) not in used_pairs:
                        intersection_count += min(counter1[w1], counter2[w2])
                        used_pairs.add((w1, w2))
                        break

            union_count = sum(counter1.values()) + sum(counter2.values()) - intersection_count

            if union_count:
                score = intersection_count / union_count

    except Exception as e:
        print(f"Erreur lors du calcul : {e}")

    return score


In [17]:
def main():
    print("===  Calculateur de similarité de phrases (Jaccard) ===")
    print("Saisissez deux phrases pour comparer leur similarité.")
    print("Tapez 'q' à tout moment pour quitter.\n")

    while True:
        # Saisie utilisateur
        s1 = input(" Entrez la première phrase : ")
        if s1.lower() == "q":
            break

        s2 = input(" Entrez la deuxième phrase : ")
        if s2.lower() == "q":
            break

        # Calcul et affichage du résultat
        score = jaccard_similarity_sentences(s1, s2)
        print(f"\n Similarité Jaccard = {score:.2f} (entre 0 et 1)\n")

        # Demander si l'utilisateur veut continuer
        again = input("Voulez-vous comparer d'autres phrases ? (o/n) : ").strip().lower()
        if again != "o":
            break

    print("\n Merci d'avoir utilisé le calculateur ! ")


if __name__ == "__main__":
    main()


===  Calculateur de similarité de phrases (Jaccard) ===
Saisissez deux phrases pour comparer leur similarité.
Tapez 'q' à tout moment pour quitter.


 Similarité Jaccard = 1.00 (entre 0 et 1)


 Merci d'avoir utilisé le calculateur ! 
